# Código redes de sentencias de la Corte Constitucional

    Autores: Andrés Felipe Patiño y Juan Carlos Rodríguez Raga
    Universidad de los Andes
    2020-20

In [1]:
import os
import requests
import urllib.parse
from bs4 import BeautifulSoup
import re
import altair as alt
import pandas as pd
from urllib.request import urlopen
from random import sample

## Extracción de sentencias

In [2]:
folder_location = './sample_sentencias'
if not os.path.exists(folder_location):
    os.mkdir(folder_location)

In [3]:
años = ['92','93','94','95','96','97','98','99','00','01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']
añosp = ['00', '01', '02', '11']
pages = ['0', '1']

In [4]:
urls=[]
for i in años:
    for p in pages:
        url= "https://www.corteconstitucional.gov.co/relatoria/radicador/buscar.php?ponente=&demandado=&Sentencia=&Tipo=Sentencias&proceso=&busqueda=&conector=AND&segundotema=&anios="+str(i)+"&pg="+str(p)+"&vs=0&accion=Buscar"
        urls.append(url)
links=[]
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all('td', attrs={'width': '100%'})
    for result in results:
        hipervinculo=result.find("a")
        if hipervinculo != None:
            link=hipervinculo.get("href")
            link_completo=urllib.parse.urljoin(url,link)
            links.append(link_completo)

In [5]:
print(len(links))

26484


In [6]:
muestra = sample(links,1022)
for link in muestra:
    filename=os.path.join(folder_location,link.split('/')[-1])
    with open(filename, 'wb') as f:
        f.write(requests.get(link).content)

In [17]:
re.search("C-322-15", str(links))
print()

<re.Match object; span=(1606550, 1606558), match='C-322-15'>

##  Extracción partes

In [7]:
files = os.listdir("./sample_sentencias/")

In [19]:
sentdct = {}
noencon = []
for i,file in enumerate(files):
    with open(str("./sample_sentencias/"+files[i]), encoding="Windows-1252") as f:   
        soup = BeautifulSoup(f, "html.parser")
        f.close()
        btags = soup.find_all("b")
        htags = soup.find_all(re.compile("h[0-9]|h"))
        etags = soup.find_all("p", class_="Estilo")
        mtags = soup.find_all("p", class_=re.compile("MsoHeading[0-9]|MsoHeading"))    
        ANT = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(btags))
        CONS = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o|", str(btags))
        RES = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(btags))
        ANT1 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(htags))
        CONS1 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(htags))
        RES1 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(htags))
        ANT2 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(etags))
        CONS2 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(etags))
        RES2 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(etags))
        ANT3 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(mtags))
        CONS3 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(mtags))
        RES3 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(mtags))
        sent = re.findall("([CcTtSs].[0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][Aa]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9][Aa]-[0-9][0-9])", str(soup))
        if sent == []:
            noencon.append("identificador %d" % (i))
        else:
            nsent = str(sent[0])
            DICTU = {"id":nsent}
            ANTt = ANT + ANT1 + ANT2 + ANT3
            CONSt = CONS + CONS1 + CONS2 + CONS3
            RESt = RES + RES1 + RES2 + RES3
            with open("./sample_sentencias/"+files[i], mode="r") as bigfile:
                reader = bigfile.read()
                if RESt != [] and CONSt != [] and ANTt != []:
                    for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    for i,part in enumerate(re.split(CONSt[0], repart1.read(), maxsplit=1)):
                        with open("CONSm_" + str(i+1), mode="w") as newfilecons:
                            newfilecons.write(CONSt[0]+part)
                    repart2 = open("CONSm_1", "r")
                    departc = open("CONSm_2", "r")
                    DICTU["CONS"] = (departc.read())
                    for i,part in enumerate(re.split(ANTt[0], repart2.read(), maxsplit=1)):
                        with open("ANTm_" + str(i+1), mode="w") as newfileant:
                            newfileant.write(ANTt[0]+part)
                    departh1 = open("ANTm_1", "r")
                    departh2 = open("ANTm_2", "r")
                    DICTU["ANT"] = (departh2.read())
                    DICTU["ENC"] = (departh1.read())
                elif RESt != [] and CONSt != [] and ANTt == []:
                    for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    for i,part in enumerate(re.split(CONSt[0], repart1.read(), maxsplit=1)):
                        with open("CONSm_" + str(i+1), mode="w") as newfilecons:
                            newfilecons.write(CONSt[0]+part)
                    repart2 = open("CONSm_1", "r")
                    departc = open("CONSm_2", "r")
                    DICTU["CONS"] = (departc.read())
                    DICTU["ENC"] = (repart2.read())
                elif RESt != [] and CONSt == [] and ANTt == []:
                    for i,part in enumerate(re.split(RESt[0], reader, maxsplit=1)):
                        with open("RESm_" + str(i+1), mode="w") as newfileres:
                            newfileres.write(RESt[0]+part)
                    repart1 = open("RESm_1", "r")
                    departr = open("RESm_2", "r")
                    DICTU["RES"] = (departr.read())
                    DICTU["CUERPO"] = (repart1.read())
                elif RESt != [] and CONSt == [] and ANTt != []:
                    print("%s_error4" % (nsent))
                else:
                    print("%s_error5" % (nsent))
            sentdct['%s' % (nsent)] = DICTU

C-088-94_error5


In [21]:
print(noencon)

['identificador 122', 'identificador 235', 'identificador 716']


In [25]:
r = open(str("./sample_sentencias/"+files[716]), encoding="Windows-1252")
print(r)

<_io.TextIOWrapper name='./sample_sentencias/T-470-13.htm' mode='r' encoding='Windows-1252'>


In [20]:
print(len(sentdct.keys()))

1019


In [ ]:
print(sentdct["su491-93"].get("CUERPO"))

In [ ]:
with open(str("./sample_sentencias/C-078-99.htm"), encoding="Windows-1252") as f:   
    soup = BeautifulSoup(f, "html.parser")
    f.close()
    btags = soup.find_all("b")
    htags = soup.find_all(re.compile("h[0-9]|h"))
    etags = soup.find_all("p", class_="Estilo")
    mtags = soup.find_all("p", class_=re.compile("MsoHeading[0-9]|MsoHeading"))    
    ANT = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(btags))
    CONS = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o|", str(btags))
    RES = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(btags))
    ANT1 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(htags))
    CONS1 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(htags))
    RES1 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(htags))
    ANT2 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(etags))
    CONS2 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(etags))
    RES2 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(etags))
    ANT3 = re.findall("ANTECEDENTES|Antecedentes|A.N.T.E.C.E.D.E.N.T.E.S|A.n.t.e.c.e.d.e.n.t.e.s|INFORMACI.N|Informaci.n", str(mtags))
    CONS3 = re.findall("CONSIDERACIONES|Consideraciones|FUNDAMENTO|Fundamento|F.U.N.D.A.M.E.N.T.O|F.u.n.d.a.m.e.n.t.o", str(mtags))
    RES3 = re.findall("DECISI.N|Decisi.n|D.E.C.I.S.I...N|D.e.c.i.s.i...|RESUELVE|Resuelve|R.E.S.U.E.L.V.E|R.e.s.u.e.l.v.e|R\nE.S.U.E.L.V.E|R\ne.s.u.e.l.v.e", str(mtags))
    sent = re.findall("([CcTtSs].[0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][Aa]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9]-[0-9][0-9]|[CcTtSs].[0-9][0-9][0-9][0-9][Aa]-[0-9][0-9])", str(soup))
    nsent = str(sent[0])
    DICTU = {"id":nsent}
    ANTt = ANT + ANT1 + ANT2 + ANT3
    CONSt = CONS + CONS1 + CONS2 + CONS3
    RESt = RES + RES1 + RES2 + RES3
    #if len(RESt) == 2:
        #RESt.pop(0)
    #elif len(RESt) == 3:
        #del RESt[0:2]

In [ ]:
print(RESt)

In [ ]:
with open(str("./sample_sentencias/C-762-09.htm"), encoding="Windows-1252") as f:   
    soup = BeautifulSoup(f, "html.parser")
    t = soup.find_all(text=True)
    print(t)